<a href="https://colab.research.google.com/github/Tstrebe2/predicting-text-difficulty/blob/dave-updates/code/dave_naive_bayes(mnb).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('drive/MyDrive/696/text_difficulty_features.csv',sep='\t')
df.head(5)

,original_text,lemmatized_text,d_chall_score,aoa_mean,aoa_min,aoa_max,conc_rating_mean,conc_rating_min,conc_rating_max,num_lemmas,label
0,There is manuscript evidence that Austen conti...,There be manuscript evidence that Austen conti...,8.533221,5.809310,3.57,12.12,2.495517,1.33,4.57,38.0,1
1,"In a remarkable comparative analysis , Mandaea...","In a remarkable comparative analysis , Mandaea...",12.320171,7.402308,2.89,11.94,2.334286,1.46,4.93,21.0,1
2,"Before Persephone was released to Hermes , who...","Before Persephone be release to Hermes , who h...",5.931500,5.231351,2.78,11.17,2.556486,1.43,4.86,40.0,1
3,Cogeneration plants are commonly found in dist...,Cogeneration plant be commonly find in distric...,7.015012,6.742000,3.56,11.53,3.369655,1.52,4.93,32.0,1
4,Geneva is the second-most-populous city in Swi...,Geneva be the second-most-populous city in Swi...,7.550745,5.455000,3.69,12.62,2.399333,1.43,4.79,22.0,1


In [4]:
df.dtypes

original_text        object
lemmatized_text      object
d_chall_score       float64
aoa_mean            float64
aoa_min             float64
aoa_max             float64
conc_rating_mean    float64
conc_rating_min     float64
conc_rating_max     float64
num_lemmas          float64
label                 int64
dtype: object

In [85]:
# Extract only the columns we'll use for the classifier
df = df[df.columns[1:-1]]

1    0.518905
0    0.481095
Name: label, dtype: float64

In [132]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split


ImportError: ignored

In [24]:
#Commented out so that the Training and Testing sets are preserved

# X = df[df.columns[:-1]]
# y = df.label

# X_train, X_test, y_train, y_test = train_test_split(X,y,
#                                                     test_size=.1,
#                                                     stratify=y,
#                                                     random_state=99)
#  Training_Set = pd.concat([X_train, pd.DataFrame(y_train)], axis=1)
#  Testing_Set = pd.concat([X_test, pd.DataFrame(y_test)], axis=1)
#  Testing_Set.to_csv('drive/Shareddrives/Milestone 2/Testing_set.csv', sep='\t', index=False)
# #  Training_Set.to_csv('drive/Shareddrives/Milestone 2/Training_set.csv', sep='\t', index=False)

In [5]:
train = pd.read_csv('drive/Shareddrives/Milestone 2/Training_set.csv', 
                      sep='\t',index_col='ix')

test = pd.read_csv('drive/Shareddrives/Milestone 2/Testing_set.csv', sep='\t',
                     index_col='ix')

In [104]:
 #Class proportions
train.label.value_counts()/ len(df)

1    0.474597
0    0.474597
Name: label, dtype: float64

In [9]:
print(f"Training set dimensions: {train.shape}")
print(f"label counts from training split (order is positive, negative): {train.label.value_counts().values}")

print(f"Testing set dimensions: {test.shape}")
print(f"label counts from training split (order is positive, negative): {test.label.value_counts().values}")


Training set dimensions: (375092, 11)
label counts from training split (order is positive, negative): [187546 187546]
Testing set dimensions: (41676, 11)
label counts from training split (order is positive, negative): [20838 20838]


In [11]:
train.head()

,original_text,lemmatized_text,d_chall_score,aoa_mean,aoa_min,aoa_max,conc_rating_mean,conc_rating_min,conc_rating_max,num_lemmas,label
ix,,,,,,,,,,,
0,There is manuscript evidence that Austen conti...,There be manuscript evidence that Austen conti...,8.817221,5.809310,3.57,12.12,2.495517,1.33,4.57,39.0,1
1,"In a remarkable comparative analysis , Mandaea...","In a remarkable comparative analysis , Mandaea...",12.320171,7.402308,2.89,11.94,2.334286,1.46,4.93,21.0,1
2,"Before Persephone was released to Hermes , who...","Before Persephone be release to Hermes , who h...",6.594628,5.231351,2.78,11.17,2.556486,1.43,4.86,42.0,1
3,Cogeneration plants are commonly found in dist...,Cogeneration plant be commonly find in distric...,7.015012,6.742000,3.56,11.53,3.369655,1.52,4.93,32.0,1
4,"Geneva -LRB- , ; , ; , ; ; -RRB- is the second...","Geneva - LRB - , ; , ; , ; ; - RRB - be the se...",9.847729,5.455000,3.69,12.62,2.399333,1.43,4.79,28.0,1


In [107]:
# X_train = train[train.columns[:-1]]
# y_train = train[train.columns[-1]]

# X_test = test[test.columns[:-1]]
# y_test = test[test.columns[-1]]

In [103]:
metrics_df = {"model_name":[],
              "model_instance":[],
              "train_accuracy": [],
              "train_precision": [],
              "train_recall": [],
              "train_f1": [],
              "train_aucprc":[],
              "train_aucroc": [],
              "test_accuracy":[],
              "test_precision":[],
              "test_f1": [],
              "test_recall":[],
              "test_aucprc":[],
              "test_aucroc": []
              }

In [29]:
from sklearn. metrics import precision_score, recall_score, precision_recall_curve, roc_auc_score, auc, accuracy_score,f1_score

In [30]:
from sklearn.metrics._plot.precision_recall_curve import plot_precision_recall_curve


In [100]:
def generate_model_name(model):
  for key,value in model.best_params_.items():
    model_name += f" {key}_{value}"
  return model_name

def compute_metrics(model,model_name,X_train_data,y_train_data, X_test_data, y_test_data):
    # extract the model's hyperparameters and make that part of the identifier for the model 

    metrics_df['model_name'].append(model_name)
    metrics_df['model_instance'].append(model)
   
    # run train data
    y_pred_train = model.predict(X_train_data)
    acc = accuracy_score(y_train_data, y_pred_train)
    recall = recall_score(y_train_data,y_pred_train)
    precision = precision_score(y_train_data, y_pred_train)
    f1 = f1_score(y_train_data,y_pred_train)
    y_proba_train = model.predict_proba(X_train_data)[:,1]
    prc,rec,thres = precision_recall_curve(y_train_data, y_proba_train)
    auc_prc = auc(rec,prc)
    roc = roc_auc_score(y_train_data, y_proba_train)
   
   
    metrics_df['train_accuracy'].append(acc)
    metrics_df['train_recall'].append(recall)
    metrics_df['train_precision'].append(precision)
    metrics_df['train_f1'].append(f1)
    metrics_df['train_aucprc'].append(auc_prc)
    metrics_df['train_aucroc'].append(roc)


    #run test data
    y_pred = model.predict(X_test_data)
    acc = accuracy_score(y_test_data, y_pred)
    recall = recall_score(y_test_data,y_pred)
    precision = precision_score(y_test_data, y_pred)
    f1 = f1_score(y_test_data,y_pred)
    y_proba = model.predict_proba(X_test_data)[:,1]
    prc,rec,thres = precision_recall_curve(y_test_data, y_proba)
    auc_prc = auc(rec,prc)
    roc = roc_auc_score(y_test_data, y_proba)

    metrics_df['test_accuracy'].append(acc)
    metrics_df['test_recall'].append(recall)
    metrics_df['test_precision'].append(precision)
    metrics_df['test_f1'].append(f1)
    metrics_df['test_aucprc'].append(auc_prc)
    metrics_df['test_aucroc'].append(roc)








In [31]:
#create a balanced sample from the training set

sample_df = train.groupby('label').apply(lambda x: x.sample(30000, random_state=99))
sample_df.label.value_counts()

X_train_samp = sample_df.iloc[:,:-1]
y_train_samp = sample_df['label']

### **Naive Bayes with Tfidf Vectorizer**

In [35]:
# NaiveBayes Classifier with Tfidf Vectorizer
from sklearn.naive_bayes import MultinomialNB, MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [36]:
# due to the amount of time it took to run the tfidf vectorizer, I manually 
# tested min_df settings and determined the optimal range to be in the 20s



pipeline = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', MultinomialNB())
                    ])

parameters = {
    'tfidf__min_df': [0,25,30],
    'tfidf__stop_words': ['english'],
    'tfidf__ngram_range': [(1, 3)],
    'clf__alpha': [1,5,10,50,100]
}

In [39]:
grid_bnb_f1 = GridSearchCV(pipeline, 
                            param_grid=parameters,
                            scoring='f1', 
                            cv=StratifiedKFold(n_splits=10),
                            return_train_score=True, 
                            n_jobs=-1).fit(X_train_samp['lemmatized_text'],y_train_samp)
grid_bnb_f1.best_params_

{'clf__alpha': 10,
 'tfidf__min_df': 0,
 'tfidf__ngram_range': (1, 3),
 'tfidf__stop_words': 'english'}

In [42]:
pd.DataFrame(grid_bnb_f1.cv_results_).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
mean_fit_time,10.455486,5.656057,5.595664,10.248847,5.584289,5.68438,10.183109,5.526847,5.605251,10.322723,5.649691,5.724106,10.429899,5.596036,5.626075
std_fit_time,0.124527,0.176214,0.060066,0.120277,0.063733,0.084922,0.135295,0.089208,0.04578,0.155679,0.063695,0.064803,0.261191,0.143465,0.126177
mean_score_time,0.504944,0.455704,0.360587,0.481726,0.370236,0.373735,0.471169,0.374427,0.363942,0.462549,0.382464,0.382776,0.488281,0.389127,0.383148
std_score_time,0.017683,0.184061,0.02176,0.013436,0.017275,0.015252,0.039073,0.011273,0.021248,0.029701,0.018329,0.013635,0.038614,0.021916,0.013035
param_clf__alpha,1,1,1,5,5,5,10,10,10,50,50,50,100,100,100
param_tfidf__min_df,0,25,30,0,25,30,0,25,30,0,25,30,0,25,30
param_tfidf__ngram_range,"(1, 3)","(1, 3)","(1, 3)","(1, 3)","(1, 3)","(1, 3)","(1, 3)","(1, 3)","(1, 3)","(1, 3)","(1, 3)","(1, 3)","(1, 3)","(1, 3)","(1, 3)"
param_tfidf__stop_words,english,english,english,english,english,english,english,english,english,english,english,english,english,english,english
params,"{'clf__alpha': 1, 'tfidf__min_df': 0, 'tfidf__...","{'clf__alpha': 1, 'tfidf__min_df': 25, 'tfidf_...","{'clf__alpha': 1, 'tfidf__min_df': 30, 'tfidf_...","{'clf__alpha': 5, 'tfidf__min_df': 0, 'tfidf__...","{'clf__alpha': 5, 'tfidf__min_df': 25, 'tfidf_...","{'clf__alpha': 5, 'tfidf__min_df': 30, 'tfidf_...","{'clf__alpha': 10, 'tfidf__min_df': 0, 'tfidf_...","{'clf__alpha': 10, 'tfidf__min_df': 25, 'tfidf...","{'clf__alpha': 10, 'tfidf__min_df': 30, 'tfidf...","{'clf__alpha': 50, 'tfidf__min_df': 0, 'tfidf_...","{'clf__alpha': 50, 'tfidf__min_df': 25, 'tfidf...","{'clf__alpha': 50, 'tfidf__min_df': 30, 'tfidf...","{'clf__alpha': 100, 'tfidf__min_df': 0, 'tfidf...","{'clf__alpha': 100, 'tfidf__min_df': 25, 'tfid...","{'clf__alpha': 100, 'tfidf__min_df': 30, 'tfid..."
split0_test_score,0.638816,0.63469,0.633689,0.672488,0.635749,0.633957,0.676915,0.642438,0.635971,0.675591,0.655477,0.652121,0.675147,0.669208,0.662822


In [43]:
grid_bnb_f1.best_params_

{'clf__alpha': 10,
 'tfidf__min_df': 0,
 'tfidf__ngram_range': (1, 3),
 'tfidf__stop_words': 'english'}

In [151]:
tfidf_prefix = 'tfidf__'
tfidf_best_params = {k[len(tfidf_prefix):]:v 
                     for (k, v) in grid_bnb_f1.best_params_.items() 
                     if tfidf_prefix in k}

In [153]:
text_features = 'lemmatized_text'

text_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(**tfidf_best_params)),
])

preprocessor = ColumnTransformer([
    ('text', text_pipe, text_features),
])


clf = Pipeline([
    ('preprocessor', preprocessor),
    ('clf', MultinomialNB(alpha=50)),
])

clf.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('text',
                                                  Pipeline(steps=[('tfidf',
                                                                   TfidfVectorizer(min_df=0,
                                                                                   ngram_range=(1,
                                                                                                3),
                                                                                   stop_words='english'))]),
                                                  'lemmatized_text')])),
                ('clf', MultinomialNB(alpha=50))])

In [154]:
model_name = "mnb alpha 50 ngram (1,3) min_df 0"
compute_metrics(clf,
                model_name,
                X_train,
                y_train,
                X_test['lemmatized_text'],
                y_test)


ValueError: ignored

In [133]:
tfidf_vec = grid_bnb_f1.best_estimator_.named_steps['tfidf']
best_tfidf_model = grid_bnb_f1.best_estimator_.named_steps['clf']
neg_tfidf_class = best_tfidf_model.feature_log_prob_[0, :].argsort()[::-1]
pos_tfidf_class = best_tfidf_model.feature_log_prob_[1, :].argsort()[::-1]

In [134]:
print("Top words in negative class:", np.take(tfidf_vec.get_feature_names_out(), neg_tfidf_class[:20]))
print("words in positive class:", np.take(tfidf_vec.get_feature_names_out(), pos_tfidf_class[:20]))

Top words in negative class: ['rrb' 'lrb' 'use' 'city' 'make' 'bear' 'commune' 'people' 'france' 'play'
 'united' 'lrb bear' 'region' 'player' 'football' 'states' 'north' 'time'
 'united states' 'know']
words in positive class: ['rrb' 'lrb' 'use' 'city' 'bear' 'know' 'france' 'year' 'include'
 'department' 'united' 'commune' 'new' 'time' 'play' 'county' 'world'
 'state' 'american' 'large']


In [135]:
# Negative Class Feature Importance by Log Probabilities
list(zip(np.take(tfidf_vec.get_feature_names_out(), neg_tfidf_class[:20]),
         [best_tfidf_model.feature_log_prob_[0,idx] for idx in neg_tfidf_class]))

[('rrb', -6.206353057167201),
 ('lrb', -6.209119548276159),
 ('use', -6.959070848281077),
 ('city', -7.010245862080496),
 ('make', -7.016677864323377),
 ('bear', -7.089617757211041),
 ('commune', -7.169213111797247),
 ('people', -7.2174507165460895),
 ('france', -7.228856548970596),
 ('play', -7.243048282711776),
 ('united', -7.263282576643848),
 ('lrb bear', -7.263848822157203),
 ('region', -7.265847923227086),
 ('player', -7.326647719846201),
 ('football', -7.327312001274062),
 ('states', -7.331309939403345),
 ('north', -7.332839987583623),
 ('time', -7.34463459069615),
 ('united states', -7.345894337473056),
 ('know', -7.353299086290129)]

In [ ]:
# Positive Class Feature Importance by Log Probabilities
list(zip(np.take(tfidf_vec.get_feature_names_out(), pos_tfidf_class[:20]),
         [best_tfidf_model.feature_log_prob_[1,idx] for idx in pos_tfidf_class]))

In [ ]:
grid_bnb_f1.best_estimator_.score(X_test['lemmatized_text'], y_test)

In [ ]:
# tf_vec = TfidfVectorizer(min_df = 30,
#                          stop_words='english',
#                          ngram_range=(1, 2))

In [ ]:
# X_train_vec = tf_vec.fit_transform(X_train['lemmatized_text'])
# X_test_vec = tf_vec.transform(X_test['lemmatized_text'])

In [ ]:
# bnb = MultinomialNB(alpha=.1).fit(X_train_vec,y_train)
# bnb.score(X_test_vec,y_test)
# manual_params = {'alpha': [0.001, 0.01, 0.1, 1,5,10,50, 100]}
# grid_bn_acc = GridSearchCV(MultinomialNB(), param_grid=param_grid)
# grid_bn_acc.fit(X_train_vec,y_train)
# grid_bn_acc.best_estimator_.score(tf_vec.transform(X_test['lemmatized_text']),y_test)

In [ ]:
model_name  = 'Tf-idf BNB alpha_100 min_df_30 ngram_1_2'

# X_test_vec = tf_vec.transform(X_test['lemmatized_text'])
compute_metrics(grid_bnb_f1.best_estimator_,
                model_name,
                X_train['lemmatized_text'],
                y_train,
                X_test['lemmatized_text'],
                y_test)



In [ ]:
metrics_df

# Mads Features

In [49]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [50]:
# X_train_mads_features = X_train.iloc[:,1:9]
# mads_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
# X_train_mads_features_imp = mads_mean.fit_transform(X_train_mads_features)

In [55]:
cols = ['d_chall_score','aoa_mean','aoa_min','aoa_max','conc_rating_mean','conc_rating_min', 'conc_rating_max','num_lemmas']
X_train_samp_mads_features = X_train_samp[:][cols]
# X_test_mads_features = X_test.iloc[:][cols]
# X_train_mads_features.describe()

In [56]:
mads_pipeline = Pipeline([('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
                          ('scaler', MinMaxScaler()),
                          ('clf', MultinomialNB())
                         ])

parameters = {
    'clf__alpha': [0.001,0.01,0.1,1,5,10,50, 100]
}

In [57]:
grid_bnb_mads_f1 = (GridSearchCV(mads_pipeline,
                                  param_grid=parameters,
                                  scoring='f1',
                                  n_jobs=-1,
                                  return_train_score=True, 
                                  cv=10).fit(X_train_samp_mads_features,y_train_samp))

In [58]:
best_model = grid_bnb_mads_f1.best_estimator_.named_steps['clf']


neg_class = best_model.feature_log_prob_[0, :].argsort()[::-1]
pos_class = best_model.feature_log_prob_[1, :].argsort()[::-1]

print(np.take(cols, neg_class[:20]))
print(np.take(cols, pos_class[:20]))





['conc_rating_max' 'aoa_max' 'd_chall_score' 'conc_rating_mean' 'aoa_mean'
 'num_lemmas' 'aoa_min' 'conc_rating_min']
['conc_rating_max' 'aoa_max' 'd_chall_score' 'conc_rating_mean' 'aoa_mean'
 'num_lemmas' 'aoa_min' 'conc_rating_min']


In [60]:
pd.DataFrame(grid_bnb_mads_f1.cv_results_).T

,0,1,2,3,4,5,6,7
mean_fit_time,0.042567,0.046819,0.040469,0.041685,0.044365,0.043202,0.041149,0.043681
std_fit_time,0.006132,0.009263,0.00194,0.00456,0.004788,0.008851,0.004077,0.006357
mean_score_time,0.009967,0.010875,0.010363,0.009921,0.009785,0.009768,0.01064,0.010206
std_score_time,0.001563,0.003684,0.003178,0.00145,0.001132,0.00077,0.00383,0.001294
param_clf__alpha,0.001,0.01,0.1,1,5,10,50,100
params,{'clf__alpha': 0.001},{'clf__alpha': 0.01},{'clf__alpha': 0.1},{'clf__alpha': 1},{'clf__alpha': 5},{'clf__alpha': 10},{'clf__alpha': 50},{'clf__alpha': 100}
split0_test_score,0.632059,0.632059,0.632059,0.632059,0.632846,0.633409,0.633663,0.636113
split1_test_score,0.617478,0.617478,0.617478,0.617478,0.617478,0.617604,0.618759,0.62069
split2_test_score,0.634043,0.634043,0.634043,0.634043,0.634487,0.634487,0.635096,0.636231
split3_test_score,0.634123,0.634123,0.634123,0.63402,0.63436,0.63436,0.634525,0.634705


In [ ]:
# Negative Class Feature Importance by Log Probabilities
list(zip(np.take(cols, neg_class[:10]),
         [best_model.feature_log_prob_[0,idx] for idx in neg_class]))

In [ ]:
# Positive Class Feature Importance by Log Probabilities
list(zip(np.take(cols, pos_class[:10]),
         [best_model.feature_log_prob_[1,idx] for idx in pos_class]))

In [ ]:
grid_bnb_mads_f1.best_estimator_

In [ ]:
X_test_mads_features

In [ ]:
grid_bnb_mads_f1.best_estimator_.score(X_train_mads_features, y_train)

In [ ]:
grid_bnb_mads_f1.best_params_

In [ ]:
model_name = 'MADS BNB alpha 10'
compute_metrics(grid_bnb_mads_f1.best_estimator_,
                model_name,
                X_train_mads_features,
                y_train,
                X_test_mads_features,
                y_test)

In [ ]:
# tfidf + Mads features

# tfidf + Mads Features

In [61]:
import numpy as np
from sklearn.compose import ColumnTransformer

In [62]:
from sklearn.pipeline import FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import KBinsDiscretizer


vectorizer = Pipeline([
              # ('select', list_trf),
              ('tfidf', TfidfVectorizer())
              ])

mads_pipe = Pipeline([
              ('impute_mean',SimpleImputer(strategy='mean')),
              ('scaler',MinMaxScaler()),
              ('bin',KBinsDiscretizer(n_bins=4))
              ])

preprocessor = ColumnTransformer(transformers=[
    ('mads',mads_pipe,cols),
    ('text', vectorizer, 'lemmatized_text')
    ])


              
pipeline = Pipeline([('preprocessor', preprocessor),
                     ('clf', MultinomialNB())  # classifier
                     ])

In [78]:
tfidf_prefix = 'preprocessor__text__tfidf__'

parameters = {
    tfidf_prefix + 'min_df': [20],
    tfidf_prefix + 'stop_words': ['english', None],
    tfidf_prefix + 'ngram_range' : [(1, 3)],
    'clf__alpha': [.0005, .001,.005, 0.01,1]
}

grid_mnb_all_f1= (GridSearchCV(pipeline,
                                  param_grid=parameters,
                                  scoring='f1',
                                  n_jobs=-1,
                                  return_train_score=True,
                                  cv=StratifiedKFold(n_splits=10))).fit(X_train_samp,y_train_samp)



/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_discretization.py:233: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 5 are removed. Consider decreasing the number of bins.
  "decreasing the number of bins." % jj


In [79]:

grid_mnb_all_f1.best_params_

{'clf__alpha': 0.01,
 'preprocessor__text__tfidf__min_df': 20,
 'preprocessor__text__tfidf__ngram_range': (1, 3),
 'preprocessor__text__tfidf__stop_words': None}

In [80]:
pd.DataFrame(grid_bnb_all_f1.cv_results_).T

,0,1,2,3,4
mean_fit_time,5.7952,6.028621,5.749536,5.720234,5.734252
std_fit_time,0.093789,0.613658,0.065293,0.053489,0.071199
mean_score_time,0.404196,0.397673,0.405262,0.407176,0.406489
std_score_time,0.007953,0.012462,0.010217,0.012835,0.014035
param_clf__alpha,0.0001,0.0005,0.001,0.005,0.01
param_preprocessor__text__tfidf__min_df,20,20,20,20,20
param_preprocessor__text__tfidf__ngram_range,"(1, 3)","(1, 3)","(1, 3)","(1, 3)","(1, 3)"
param_preprocessor__text__tfidf__stop_words,english,english,english,english,english
params,"{'clf__alpha': 0.0001, 'preprocessor__text__tf...","{'clf__alpha': 0.0005, 'preprocessor__text__tf...","{'clf__alpha': 0.001, 'preprocessor__text__tfi...","{'clf__alpha': 0.005, 'preprocessor__text__tfi...","{'clf__alpha': 0.01, 'preprocessor__text__tfid..."
split0_test_score,0.677633,0.677633,0.677633,0.677633,0.677846


In [93]:
# Get train/val split for best model parameters
text_features= ['lemmatized_text']
num_features = ['d_chall_score', 'aoa_mean',
       'aoa_min', 'aoa_max', 'conc_rating_mean', 'conc_rating_min',
       'conc_rating_max', 'num_lemmas']

y = train['label']
X_train, X_val, y_train, y_val = train_test_split(train[X_features],
                                                  y,
                                                  stratify=y,
                                                  test_size=.1,
                                                  random_state=99)


In [95]:
tfidf_prefix = 'preprocessor__text__tfidf__'
clf_prefix = 'clf__'

tfidf_best_params = {k[len(tfidf_prefix):]:v 
                     for (k, v) in grid_bnb_all_f1.best_params_.items() 
                     if tfidf_prefix in k}

clf_best_params = {k[len(clf_prefix):]:v 
                   for (k, v) in grid_bnb_all_f1.best_params_.items() 
                   if clf_prefix in k}


num_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('binner', KBinsDiscretizer(n_bins=4)),
])

text_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(**tfidf_best_params)),
])

preprocessor = ColumnTransformer([
    ('num', num_pipe, num_features),
    ('text', text_pipe, 'lemmatized_text'),
])

clf = Pipeline([
    ('preprocessor', preprocessor),
    ('clf', MultinomialNB(**clf_best_params)),
])

In [96]:
clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_discretization.py:233: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 5 are removed. Consider decreasing the number of bins.
  "decreasing the number of bins." % jj


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('binner',
                                                                   KBinsDiscretizer(n_bins=4))]),
                                                  ['d_chall_score', 'aoa_mean',
                                                   'aoa_min', 'aoa_max',
                                                   'conc_rating_mean',
                                                   'conc_rating_min',
                                                   'conc_rating_max',
                                                   'num_lemmas']),
                                                 ('text',
                                                  Pipeline(steps=[('tfidf',
        

In [104]:
# compute metrics from the test split data

model_name = "Multi Naive Bayes"
for k,v in grid_mnb_all_f1.best_params_.items():
  model_name +=f' {v}'

compute_metrics(clf,model_name,X_train, y_train, X_val, y_val)

In [105]:
metrics_df = pd.DataFrame(metrics_df)

In [107]:
metrics_df.T

,0
model_name,"Multi Naive Bayes 0.01 20 (1, 3) None"
model_instance,"(ColumnTransformer(transformers=[('num',\n ..."
train_accuracy,0.681192
train_precision,0.667565
train_recall,0.721851
train_f1,0.693648
train_aucprc,0.755558
train_aucroc,0.752685
test_accuracy,0.663876
test_precision,0.651965


In [131]:

num_features_trf = (clf['preprocessor']
                    .transformers_[0][1]
                    ['binner']
                    .get_feature_names_out(num_features))

text_features_trf = (clf['preprocessor']
                 .transformers_[1][1]
                 ['tfidf']
                 .get_feature_names_out())

all_features_trf = list(num_features_trf) + list(text_features_trf)


# neg_class = clf['clf'].feature_log_prob_[0, :].argsort()[::-1]
# pos_class = clf['clf'].feature_log_prob_[ :, :].argsort()[::-1]

# np.take(all_features_trf, pos_class[:20])





num_features_trf



# neg_class = clf['clf'].feature_log_prob_[0, :].argsort()[::-1]
# pos_class = clf['clf'].feature_log_prob_[1, :].argsort()[::-1]

# Negative Class Feature Importance by Log Probabilities


# neg_lst = list(zip(np.take(tfidf_feature_names, neg_class[:20]),
#                    (np.take(clf['clf'].feature_log_prob_[0,:],neg_class))))
         
         
      

# Positive Class Feature Importance by Log Probabilities
# pos_lst = list(zip(np.take(ttfidf_feature_names, pos_class[:20]),
#          [clf['clf'].feature_log_prob_[1,idx] for idx in pos_class]))






array(['d_chall_score_0.0', 'd_chall_score_1.0', 'd_chall_score_2.0',
       'd_chall_score_3.0', 'aoa_mean_0.0', 'aoa_mean_1.0',
       'aoa_mean_2.0', 'aoa_mean_3.0', 'aoa_min_0.0', 'aoa_min_1.0',
       'aoa_min_2.0', 'aoa_min_3.0', 'aoa_max_0.0', 'aoa_max_1.0',
       'aoa_max_2.0', 'aoa_max_3.0', 'conc_rating_mean_0.0',
       'conc_rating_mean_1.0', 'conc_rating_mean_2.0',
       'conc_rating_mean_3.0', 'conc_rating_min_0.0',
       'conc_rating_min_1.0', 'conc_rating_min_2.0',
       'conc_rating_max_0.0', 'conc_rating_max_1.0',
       'conc_rating_max_2.0', 'conc_rating_max_3.0', 'num_lemmas_0.0',
       'num_lemmas_1.0', 'num_lemmas_2.0', 'num_lemmas_3.0'], dtype=object)

,lemmatized_text,d_chall_score,aoa_mean,aoa_min,aoa_max,conc_rating_mean,conc_rating_min,conc_rating_max,num_lemmas
ix,,,,,,,,,
323495,"Vortex - LRB - album - RRB - , a remix album b...",12.727511,7.742857,2.89,12.94,3.105000,1.46,4.69,9.0
136836,The Copts of Egypt claim to have first make th...,5.278911,5.168667,2.89,9.74,2.411875,1.33,4.90,18.0
362034,Toni Michelle Braxton - LRB - bear October 7 1...,12.045000,5.128571,3.58,7.72,3.223333,1.46,4.88,20.0
145925,"Othello , the Moor of Venice be a tragedy by W...",10.275800,5.190000,2.89,11.80,2.358889,1.43,4.22,48.0
258440,Official,15.839600,8.680000,8.68,8.68,NaN,NaN,NaN,1.0


# Sentence Embeddings

array(['00', '000', '000 000', ..., 'ôte azur region', 'ôte dazur',
       'ürttemberg'], dtype=object)

In [ ]:
from numpy import loadtxt
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [ ]:
embeddings = loadtxt('drive/Shareddrives/Milestone 2/embeddings.csv', delimiter=',')
embeddings

In [ ]:
embeddings.shape

In [ ]:
# Combine the embeddings and the original_text + Mads features
emb = pd.DataFrame(embeddings)
new_df = pd.concat([df,emb], axis=1)
new_df.head()

In [ ]:
#Extract training
X_train_indexes = X_train.index
X_train_emb = new_df.iloc[X_train_indexes, 11:]



# Extract just the embedding data from test set
X_test_indexes = X_test.index
X_test_emb = new_df.iloc[X_test_indexes, 11:]

X_train_emb.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
embed_pipeline = Pipeline([('scaler', MinMaxScaler()),
                          ('clf',MultinomialNB())
                         ])

parameters = {
    'clf__alpha': [0.0001,0.001,0.01,0.1,1,5,10,50, 100,200,500]
}


grid_bnb_emb_f1 = (GridSearchCV(embed_pipeline,
                                 param_grid=parameters,
                                 scoring='f1',
                                 n_jobs=-1, 
                                 cv=10).fit(X_train_emb,y_train))


In [ ]:
grid_bnb_emb_f1.best_params_

In [ ]:
grid_bnb_emb_f1.best_estimator_.score(X_train_emb,y_train)

In [ ]:
model_name = 'embedding BNB alpha .0001'
# X_test_emb_mm = scaler.transform(X_test_emb)
compute_metrics(grid_bnb_emb_f1.best_estimator_,
                model_name,
                X_train_emb,
                y_train,
                X_test_emb,
                y_test)

# Sentence Embeddings + Mads Feaures

In [ ]:
model_name = "emb + mads"

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler, Normalizer

In [ ]:
X_train_emb_mads = new_df.iloc[X_train.index].copy()
X_test_emb_mads = new_df.iloc[X_test.index].copy()

X_train_emb_mads.drop(['original_text','lemmatized_text','label'],inplace=True,axis=1)
X_test_emb_mads.drop(['original_text','lemmatized_text','label'],inplace=True,axis=1)


In [ ]:

mads_pipe = Pipeline([
              ('impute_mean',SimpleImputer(strategy='mean')),
              ])

preprocessor = ColumnTransformer(transformers=[
    ('mads',mads_pipe,cols)
    ])


              
pipeline = Pipeline([('preprocessor', preprocessor),
                     ('scaler',MinMaxScaler()),
                     ('clf', MultinomialNB())  # classifier
                     ])


param_grid = {
    'clf__alpha': [0.001, 0.01, 0.1,1,2,3,4,5,10,50, 100,500]
}

In [ ]:
grid_bnb_embmads_f1 = GridSearchCV(pipeline,
                                    param_grid=param_grid,
                                    scoring='f1',
                                    n_jobs=-1,
                                    cv=10)
grid_bnb_embmads_f1.fit(X_train_emb_mads,y_train)

In [ ]:
grid_bnb_embmads_f1.best_params_

In [ ]:
grid_bnb_embmads_f1.best_estimator_.score(X_train_emb_mads,y_train)

In [ ]:
model_name += ' alpha 10'
compute_metrics(grid_bnb_embmads_f1.best_estimator_,model_name,X_train_emb_mads, y_train, X_test_emb_mads, y_test)

In [ ]:
#Earlier Code (pre-pipeline)

# X_train_emb_mads = np.hstack((X_train_emb_mm,
#                               X_train_mads_features))

# param_grid = {'alpha': [0.001, 0.01,0.1,.5,1,2,3,4,5,10,25,50,100,500]}
# grid_mnb_embmads_acc = GridSearchCV(MultinomialNB(), param_grid=param_grid)
# grid_mnb_embmads_acc.fit(X_train_emb_mads,y_train)

# grid_mnb_embmads_acc.best_params_

# Confirm the number of nan values based on the conc_rating_mean and fill with 0 
# X_test_mads_features[X_test_mads_features['conc_rating_mean'].isna()==True]
# X_test_mads_features.fillna(0,inplace=True)

# model_name = 'embedding NB alpha .001'
# compute_metrics(grid_mnb_embmads_acc.best_estimator_,model_name,X_test_emb_mads, y_test)

In [ ]:
metrics = pd.DataFrame(metrics_df)

In [ ]:
metrics

In [135]:
import joblib

In [136]:
model_path = ''.join(['drive/Shareddrives/Milestone 2/multinomialnaivebayes-clf.joblib'])
joblib.dump(clf['clf'], model_path)

['drive/Shareddrives/Milestone 2/multinomialnaivebayes-clf.joblib']

In [ ]:
f = open('drive/Shareddrives/Milestone 2/mnnb_model.pkl','wb')
pickle.dump(best_model,f)